### 문제 3-1 : 콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [8]:
import os
import time
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pprint import pprint

In [9]:
def setup_environment():
    if not load_dotenv():
        print("'.env' 파일을 찾을 수 없습니다. 환경 변수를 수동으로 설정해주세요.")
        return False
    
    if "UPSTAGE_API_KEY" not in os.environ:
        print("UPSTAGE_API_KEY가 .env 파일에 없습니다. 키를 추가해주세요.")
        return False
        
    print("API 키가 성공적으로 로드되었습니다.")
    return True

In [10]:
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

In [12]:
def main():
    if not setup_environment():
        return

    print("\n--- 0단계: 문서 로드 ---")
    start_time = time.time()
    try:
        loader = PyPDFLoader("/Users/eon/skRookies/myLangChain/mylangchain-app/src/mylangchain_app/data/콘텐츠분쟁해결_사례.pdf")
        documents = loader.load()
        print(f"PDF 문서 로딩 완료. 총 {len(documents)} 페이지. (소요 시간: {time.time() - start_time:.2f}초)")
    except Exception as e:
        print(f"PDF 로딩 중 오류 발생: {e}")
        return

    print("\n--- 1단계: 문서 분할 ---")
    start_time = time.time()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        separators=[
            "\n【사건개요】",
            "\n【쟁점사항】",
            "\n【처리경위】",
            "\n【처리결과】",
            "\n■", "\n\n", "\n", ".", " ", ""
        ]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"문서 분할 완료. 총 {len(split_docs)}개의 청크 생성. (소요 시간: {time.time() - start_time:.2f}초)")

    print("\n--- 2단계: 임베딩 모델 설정 ---")
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    print(f"임베딩 모델 '{embeddings.model}'이 설정되었습니다.")

    print("\n--- 3단계: 벡터 DB 생성 및 검색기 설정 ---")
    start_time = time.time()
    vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    print(f"FAISS 벡터 DB 및 검색기 생성 완료. (소요 시간: {time.time() - start_time:.2f}초)")

    print("\n--- 4단계: LLM 설정 ---")
    llm = ChatUpstage(
        model="solar-pro",
        temperature=0.2
    )
    print(f"LLM '{llm.model_name}'이 설정되었습니다.")

    print("\n--- 5단계: 프롬프트 템플릿 작성 ---")
    prompt_template = """
    당신은 콘텐츠 분야 전문 법률 자문가입니다. 
    아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해 주세요.

    관련 분쟁사례:
    {context}

    상담 내용: {question}

    답변 가이드라인:
    1. 제시된 사례들을 근거로 답변하세요
    2. 관련 법령이나 조항이 있다면 명시하세요
    3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
    4. 실무적 해결방안을 단계별로 제시하세요
    5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

    전문 법률 조언:"""
    
    prompt = PromptTemplate.from_template(prompt_template)
    print("법률 자문 프롬프트가 준비되었습니다.")

    # 6단계: QA 체인 생성
    print("\n--- 6단계: RAG QA 체인 생성 ---")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    print("RetrievalQA 체인이 성공적으로 생성되었습니다.")

    print("\n--- 7 & 8단계: 테스트 질문 실행 및 분쟁 유형 분류 ---")
    test_questions = [
        "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
        "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
        "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
        "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
        "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
    ]

    for i, question in enumerate(test_questions):
        print(f"\n{'='*20} 테스트 질문 {i+1} {'='*20}")
        
        # 8단계: 분쟁 유형 분류
        dispute_type = classify_dispute_type(question)
        print(f"질문: {question}")
        print(f"예상 분쟁 유형: {dispute_type}")
        print("-" * 50)
        
        # QA 체인 실행
        start_time = time.time()
        result = qa_chain.invoke({"query": question})
        end_time = time.time()

        print(f"[전문 법률 조언 답변] (생성 시간: {end_time - start_time:.2f}초)")
        pprint(result["result"])

        print("\n[참조된 원본 문서]")
        # 참조된 소스 문서들의 출처와 내용 일부를 출력
        unique_sources = {}
        for doc in result["source_documents"]:
            source_page = f"페이지: {doc.metadata.get('page', 'N/A')}"
            if source_page not in unique_sources:
                 unique_sources[source_page] = doc.page_content[:200] + "..."
        
        for source, content in unique_sources.items():
            print(f"- {source}")
            # print(f"  내용: {content}") # 더 자세한 내용을 보고 싶으면 주석 해제

    print(f"\n{'='*20} 모든 테스트 완료 {'='*20}")

if __name__ == "__main__":
    main()


API 키가 성공적으로 로드되었습니다.

--- 0단계: 문서 로드 ---
PDF 문서 로딩 완료. 총 109 페이지. (소요 시간: 1.57초)

--- 1단계: 문서 분할 ---
문서 분할 완료. 총 104개의 청크 생성. (소요 시간: 0.00초)

--- 2단계: 임베딩 모델 설정 ---
임베딩 모델 'solar-embedding-1-large'이 설정되었습니다.

--- 3단계: 벡터 DB 생성 및 검색기 설정 ---
FAISS 벡터 DB 및 검색기 생성 완료. (소요 시간: 15.29초)

--- 4단계: LLM 설정 ---
LLM 'solar-pro'이 설정되었습니다.

--- 5단계: 프롬프트 템플릿 작성 ---
법률 자문 프롬프트가 준비되었습니다.

--- 6단계: RAG QA 체인 생성 ---
RetrievalQA 체인이 성공적으로 생성되었습니다.

--- 7 & 8단계: 테스트 질문 실행 및 분쟁 유형 분류 ---

==================== 테스트 질문 1 ====================
질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
예상 분쟁 유형: 게임
--------------------------------------------------
[전문 법률 조언 답변] (생성 시간: 9.09초)
('### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안  \n'
 '\n'
 '#### 1. **사례 분석을 통한 법적 쟁점**  \n'
 '제시된 사례(2006, 2009년 시스템 오류 사례 등)에 따르면, 게임사의 시스템 오류로 인한 아이템 복구 여부는 다음 요소에 따라 '
 '결정됩니다:  \n'
 '- **(1) 오류 인정 여부**: 게임사가 시스템 오류를 공식적으로 인정하는지 (예: 2006년 사례에서는 복구 수용, 2009년 '
 '사례에서는 오류 부인).  \n'
 '- **(2) 계정 소유권**: 신청인